<a href="https://colab.research.google.com/github/giovannaFantacini/ReconhecimentoDeepFake/blob/main/deepFakeProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio Pillow facenet-pytorch==2.5.2 torch==1.11.0 opencv-python grad-cam

In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import os
import numpy as np
from PIL import Image
import zipfile
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/DeepFakeDetection/examples.zip","r") as zip_ref:
    zip_ref.extractall(".")

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

checkpoint = torch.load("/content/drive/MyDrive/DeepFakeDetection/resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()


In [ ]:
def predict(input_image:Image.Image, true_label:str):
    """Predict the label of the input_image"""
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    face = face.unsqueeze(0) # add the batch dimension
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

    # convert the face into a numpy array to be able to plot it
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

    target_layers=[model.block8.branch1[-1]]
    use_cuda = True if torch.cuda.is_available() else False
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=use_cuda)
    targets = [ClassifierOutputTarget(0)]

    grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
    face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < 0.5 else "fake"

        real_prediction = 1 - output.item()
        fake_prediction = output.item()

        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }
    return confidences, face_with_mask

In [ ]:
import os

def list_images_with_labels(directory_path):
    real_images = []
    fake_images = []

    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            label = os.path.basename(root)

            if label == 'real':
                real_images.append([file_path])
            elif label == 'fake':
                fake_images.append([file_path])

    return real_images, fake_images

In [ ]:
real_images, fake_images = list_images_with_labels('/content/drive/MyDrive/DeepFakeDetection/images')

with gr.Blocks() as demo:

  with gr.Row():
    with gr.Column():
      inputImage = gr.Image(label="Input Image", type="pil")
      clear = gr.ClearButton()

    with gr.Column():
      outputLabel = gr.Label(label="Class", scale=2)
      outputImage = gr.Image(label="Face with Explainability", type="pil", interactive=False, min_width=50)

  # examples_list= gr.Examples(inputs=[[examples[i]["path"], examples[i]["label"]] for i in range(10)])
  examples_real= gr.Examples(examples=real_images, inputs=[inputImage], outputs=[outputLabel, outputImage], fn=predict, label="Real Images", run_on_click=True)
  examples_fake= gr.Examples(examples=fake_images, inputs=[inputImage], outputs=[outputLabel, outputImage], fn=predict, label="Fake Images", run_on_click=True)
  clear.add([inputImage, outputLabel, outputImage])
  inputImage.upload(fn=predict, inputs=[inputImage], outputs=[outputLabel, outputImage])

demo.launch(share=True)